<a href="https://colab.research.google.com/github/divyansh-shrotriya/oral-cancer-cnn/blob/main/Oral_Cancer_InceptionResNetV2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Dataset Reference

This study uses the following publicly available dataset:

Rahman, Tabassum Yesmin (2019). *A histopathological image repository of normal epithelium of Oral Cavity and Oral Squamous Cell Carcinoma*.  
Mendeley Data, Version 1.  
DOI: https://doi.org/10.17632/ftmp4cvtmb.1


Mount Google Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


GPU Check

In [1]:
import tensorflow as tf
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

Imports

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import InceptionResNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    classification_report
)

Dataset Path & Hyperparameters

In [2]:
DATASET_PATH = "/content/drive/MyDrive/First_Set"

IMAGE_SIZE = (224, 224)
BATCH_SIZE = 8        # small batch → less overfitting
EPOCHS = 30           # early stopping will cut earlier
NUM_CLASSES = 2

Data Generators (EDA-informed)

In [3]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=30,
    zoom_range=0.25,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    DATASET_PATH,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    subset="training",
    shuffle=True
)

val_generator = train_datagen.flow_from_directory(
    DATASET_PATH,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    subset="validation",
    shuffle=False
)

test_generator = test_datagen.flow_from_directory(
    DATASET_PATH,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=False
)

Found 424 images belonging to 2 classes.
Found 104 images belonging to 2 classes.
Found 528 images belonging to 2 classes.


Class Label Mapping

In [4]:
class_labels = list(train_generator.class_indices.keys())
class_labels

['100x_Normal_Oral_Cavity_Histopathological_Images',
 '100x_OSCC_Histopathological_Images']

Load InceptionResNetV2 (Transfer Learning)

In [5]:
base_model = InceptionResNetV2(
    weights="imagenet",
    include_top=False,
    input_shape=(224, 224, 3)
)

# Freeze backbone (critical for small dataset)
for layer in base_model.layers:
    layer.trainable = False

219055592/219055592 ━━━━━━━━━━━━━━━━━━━━ 6s 0us/step


Custom Classification Head

In [6]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation="relu")(x)
x = Dropout(0.5)(x)
x = Dense(64, activation="relu")(x)
x = Dropout(0.3)(x)

output = Dense(NUM_CLASSES, activation="softmax")(x)

model = Model(inputs=base_model.input, outputs=output)


Compile Model

In [7]:
model.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 111, 111,  │        864 │ input_layer[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 111, 111,  │         96 │ conv2d[0][0]      │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation          │ (None, 111, 111,  │          0 │ batch_normalizat… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 109, 109,  │      9,216 │ activation[0][0]  │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 109, 109,  │         96 │ conv2d_1[0][0]    │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_1        │ (None, 109, 109,  │          0 │ batch_normalizat… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 109, 109,  │     18,432 │ activation_1[0][… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 109, 109,  │        192 │ conv2d_2[0][0]    │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_2        │ (None, 109, 109,  │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 54, 54,    │          0 │ activation_2[0][… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 54, 54,    │      5,120 │ max_pooling2d[0]… │
│                     │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 54, 54,    │        240 │ conv2d_3[0][0]    │
│ (BatchNormalizatio… │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_3        │ (None, 54, 54,    │          0 │ batch_normalizat… │
│ (Activation)        │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 52, 52,    │    138,240 │ activation_3[0][… │
│                     │ 192)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 52, 52,    │        576 │ conv2d_4[0][0]    │
│ (BatchNormalizatio… │ 192)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_4        │ (None, 52, 52,    │          0 │ batch_normalizat

 Total params: 54,746,786 (208.84 MB)

 Trainable params: 410,050 (1.56 MB)

 Non-trainable params: 54,336,736 (207.28 MB)

Callbacks (Overfitting Control)

In [8]:
callbacks = [
    EarlyStopping(monitor="val_loss", patience=6, restore_best_weights=True),
    ModelCheckpoint(
        "/content/drive/MyDrive/oral_cancer_best.keras",
        monitor="val_loss",
        save_best_only=True
    )
]

Train Model

In [9]:
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=EPOCHS,
    callbacks=callbacks
)

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/30
53/53 ━━━━━━━━━━━━━━━━━━━━ 389s 7s/step - accuracy: 0.8238 - loss: 0.5751 - val_accuracy: 0.8462 - val_loss: 0.3717
Epoch 2/30
53/53 ━━━━━━━━━━━━━━━━━━━━ 40s 755ms/step - accuracy: 0.8042 - loss: 0.4785 - val_accuracy: 0.8365 - val_loss: 0.4027
Epoch 3/30
53/53 ━━━━━━━━━━━━━━━━━━━━ 42s 793ms/step - accuracy: 0.8119 - loss: 0.5103 - val_accuracy: 0.8365 - val_loss: 0.3662
Epoch 4/30
53/53 ━━━━━━━━━━━━━━━━━━━━ 45s 844ms/step - accuracy: 0.7976 - loss: 0.5173 - val_accuracy: 0.8365 - val_loss: 0.3598
Epoch 5/30
53/53 ━━━━━━━━━━━━━━━━━━━━ 44s 834ms/step - accuracy: 0.8259 - loss: 0.4734 - val_accuracy: 0.8365 - val_loss: 0.3190
Epoch 6/30
53/53 ━━━━━━━━━━━━━━━━━━━━ 44s 834ms/step - accuracy: 0.8223 - loss: 0.5006 - val_accuracy: 0.8750 - val_loss: 0.2849
Epoch 7/30
53/53 ━━━━━━━━━━━━━━━━━━━━ 41s 773ms/step - accuracy: 0.8177 - loss: 0.4583 - val_accuracy: 0.8462 - val_loss: 0.2918
Epoch 8/30
53/53 ━━━━━━━━━━━━━━━━━━━━ 39s 743ms/step - accuracy: 0.8303 - loss: 0.4753 - val_accura

Utility Function for Metrics

In [10]:
def evaluate_model(generator, model):
    generator.reset()
    preds = model.predict(generator)
    y_pred = np.argmax(preds, axis=1)
    y_true = generator.classes

    return {
        "Accuracy": accuracy_score(y_true, y_pred),
        "Precision": precision_score(y_true, y_pred, average="weighted"),
        "Recall": recall_score(y_true, y_pred, average="weighted"),
        "F1-score": f1_score(y_true, y_pred, average="weighted")
    }

Training Metrics

In [11]:
train_metrics = evaluate_model(train_generator, model)
train_metrics

53/53 ━━━━━━━━━━━━━━━━━━━━ 43s 577ms/step


{'Accuracy': 0.7806603773584906,
 'Precision': 0.7296124558081556,
 'Recall': 0.7806603773584906,
 'F1-score': 0.7506394050599141}

Validation Metrics

In [12]:
val_metrics = evaluate_model(val_generator, model)
val_metrics

13/13 ━━━━━━━━━━━━━━━━━━━━ 7s 554ms/step


{'Accuracy': 0.9038461538461539,
 'Precision': 0.9025865946918579,
 'Recall': 0.9038461538461539,
 'F1-score': 0.8911665257819105}

Test Metrics

In [13]:
test_metrics = evaluate_model(test_generator, model)
test_metrics

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


66/66 ━━━━━━━━━━━━━━━━━━━━ 33s 501ms/step


{'Accuracy': 0.8901515151515151,
 'Precision': 0.8851266065789347,
 'Recall': 0.8901515151515151,
 'F1-score': 0.8869216395715652}

Metrics Summary Table

In [14]:
import pandas as pd

metrics_df = pd.DataFrame.from_dict(
    {
        "Training": train_metrics,
        "Validation": val_metrics,
        "Test": test_metrics
    },
    orient="index"
)

metrics_df

,Accuracy,Precision,Recall,F1-score
Training,0.780660,0.729612,0.780660,0.750639
Validation,0.903846,0.902587,0.903846,0.891167
Test,0.890152,0.885127,0.890152,0.886922
